### Pasqal QUBO

In [1]:
# https://github.com/pasqal-io/Pulser/blob/develop/tutorials/applications/QAOA%20and%20QAA%20to%20solve%20a%20QUBO%20problem.ipynb

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# from pulser import Pulse, Sequence, Register
# from pulser_simulation import Simulation
# from pulser.devices import Chadoq2
# from pulser.waveforms import InterpolatedWaveform

from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

In [3]:
Q = np.array(
    [
        [-10.0,      19.7365809,  19.7365809,  5.42015853, 5.42015853],
        [19.7365809, -10.0,       20.67626392, 0.17675796, 0.85604541],
        [19.7365809, 20.67626392, -10.0,       0.85604541, 0.17675796],
        [5.42015853, 0.17675796,  0.85604541,  -10.0,      0.32306662],
        [5.42015853, 0.85604541,  0.17675796,  0.32306662, -10.0     ],
    ]
)

In [4]:
# Classical Brute Force

bitstrings = [np.binary_repr(i, len(Q)) for i in range(len(Q) ** 2)]
costs = []
for (
    b
) in bitstrings:  # this takes exponential time with the dimension of the QUBO
    z = np.array(list(b), dtype=int)
    cost = z.T @ Q @ z
    costs.append(cost)
zipped = zip(bitstrings, costs)
sort_zipped = sorted(zipped, key=lambda x: x[1])
print(sort_zipped[:3])

[('01011', -27.288260020000003), ('00111', -27.288260019999996), ('00101', -19.64648408)]
